In [1]:
%load_ext autoreload

In [2]:
import sys
sys.path.append("..") #this is to add the above folder to the package directory
import os
#import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pandas import ExcelWriter

# from pandas.plotting import register_matplotlib_converters
import plotly.express as px
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline() #this will allow you to use cufflinks offline
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
%matplotlib inline 
init_notebook_mode(connected=True) #this will connect plotly to the java script

### Reading the file and seting the dataframe


In [24]:
folder_path = os.path.join('..','data')

# Fierza Reservoir:
fier_curve = pd.read_excel(os.path.join(folder_path, 'level_volume_curves.xlsx'),sheet_name='fierza') #This file shows the V-E curve for the reservoirs
df_hisopr_fr = pd.read_excel(os.path.join(folder_path, 'New rules Fierza 20210401.xlsx'),sheet_name='opr_measure') #This dataframe represnts actual data measured by operatores
df_newopr_fr = pd.read_excel(os.path.join(folder_path, 'New rules Fierza 20210401.xlsx'),sheet_name='weekly') #This dataframe represnts modeling data. Becarful when you compare it with the other dataframe.

# Spilje Reservoir:
spil_curve = pd.read_excel(os.path.join(folder_path, 'level_volume_curves.xlsx'),sheet_name='spilje') #This file shows the V-E curve for the reservoirs
df_hisopr_sp = hist_rules_sp = pd.read_excel(os.path.join(folder_path, 'Shpilje_historical_rules2021416.xlsx'))
df_newopr_sp = df_opr_sp = pd.read_excel(os.path.join(folder_path, 'New rules Shpilje 20210305.xlsx'),sheet_name='weekly') #This dataframe represnts modeling data. Becarful when you compare it with the other dataframe.



## 1. Fierza reservoir calculation:


In [25]:
# processing the historical dataframe to get weekly flows:

df_hisopr_fr['year'] = df_hisopr_fr['Date'].dt.year 
df_hisopr_fr['month'] = df_hisopr_fr['Date'].dt.month
df_hisopr_fr['week'] = df_hisopr_fr['Date'].dt.week
df_hisopr_fr2=df_hisopr_fr.groupby(['week'])['water level [m]', 'inflow [m3/s]', 'inflow [m3/s].1',
       'spillway outflow [milion m3]'].mean().reset_index()
df_hisopr_fr2 = df_hisopr_fr2[df_hisopr_fr2.week != 53]

<ipython-input-25-9e05d2fba616>:5: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.



,Date,water level [m],inflow [m3/s],inflow [m3/s].1,spillway outflow [milion m3],year,month,week
0,2003-01-01,283.84,168,179.0,0.0,2003,1,1
1,2003-01-02,283.72,160,168.0,0.0,2003,1,1
2,2003-01-03,284.30,223,224.0,0.0,2003,1,1
3,2003-01-04,285.00,535,549.0,0.0,2003,1,1
4,2003-01-05,286.15,600,683.0,0.0,2003,1,1


In [36]:
# merging two the historical rules with the new rules dataframes:
dff_fr = pd.merge(df_hisopr_fr2, df_newopr_fr, on='week')
dff_fr.reset_index(inplace=True)
dff_fr['lower_level']=176
#dff_fr.head()

In [30]:
# Flood Protection Scenario

levels = [5, 10, 15, 20]
Fierza_power = 500
density = 1000 #(kg/m3) = 
g = 9.81 #(m/sec2) 
Turb_eff = 0.9 #
#fierza_lower_level = 176

for level in levels:  
    # Create the new dataframe
    
    _df = dff_fr[['month', 'week', 'max_level(m)','hist_level(m)', 'max_storage(MCM)', 'hist_storage(MCM)','hist_buffer(MCM)','lower_level']].copy()
        
    # Calculating new buffer and storage
    _df['new_buffer(MCM)'] = _df['hist_buffer(MCM)'] + (_df['hist_buffer(MCM)']*(level/100))
    _df.loc[(_df['month']>4) & (_df['month']<10), 'new_buffer(MCM)'] = _df['hist_buffer(MCM)']
    
    _df['new_storage(MCM)'] = _df['max_storage(MCM)'] - _df['new_buffer(MCM)']
    
    _df['changeinbuffer(MCM)'] = (_df['new_buffer(MCM)'] - _df['hist_buffer(MCM)'])/_df['hist_buffer(MCM)']
    _df['additional_buffer(MCM)'] = (_df['new_buffer(MCM)'] - _df['hist_buffer(MCM)'])
    
    # Reading the new water level based on the volume-elevation curve
    level_predict = np.poly1d(np.polyfit(fier_curve['Volume(MCM)'], fier_curve['Level(m)'], 2))
    _df['new_level(m)'] = level_predict(_df['new_storage(MCM)'].round(0))
    _df.loc[(_df['month']>4) & (_df['month']<10), 'new_level(m)'] = _df['hist_level(m)']
    
    #_df['lowerlevel(m)'] = fierza_lower_level
    # Add the calculations of X for each week
    _df['hist_head(m)'] = _df['hist_level(m)'] - _df['lowerlevel(m)']
    _df['hist_Q(m/sec)'] = ((Fierza_power*1000000)/(density*g*Turb_eff*_df['hist_head(m)']))
    _df['hist_flow(MCM)'] = (_df['hist_Q(m/sec)']*3600*8760)/1000000
    _df['hist_energy(PJ)'] = ((Fierza_power/1000)*31.536)
    _df['hist_X(MCM/PJ)'] = _df['hist_flow(MCM)']/_df['hist_energy(PJ)']
    
    _df['new_head(m)'] = _df['new_level(m)'] - _df['lowerlevel(m)']
    _df['new_Q(m/sec)'] = ((Fierza_power*1000000)/(density*g*Turb_eff*_df['new_head(m)']))
    _df['new_flow(MCM)'] = (_df['new_Q(m/sec)']*3600*8760)/1000000
    _df['new_energy(PJ)'] = ((Fierza_power/1000)*31.536)
    _df['new_X(MCM/PJ)'] = _df['new_flow(MCM)']/_df['new_energy(PJ)']
    _df['mode'] = 0
    # Generate the outputfile that goes to osemosys
    hist_cap = _df[['month','week','hist_storage(MCM)']]
    new_cap = _df[['month','week','new_storage(MCM)']]
    hist_x = _df[['week','mode','hist_X(MCM/PJ)']]
    new_x = _df[['week','mode','new_X(MCM/PJ)']]
    rule = _df.groupby('month')[['hist_level(m)','new_level(m)','additional_buffer(MCM)']].mean().round(1).reset_index()
    rule['diff'] = rule['hist_level(m)'] - rule['new_level(m)']
    
    #res_cap = ref[['week','Utilized Volume (MCM)']]
    for i in range(20,56):
        hist_cap['20{}'.format(i)]=hist_cap['hist_storage(MCM)']
        new_cap['20{}'.format(i)]=new_cap['new_storage(MCM)']
    hist_cap.drop('hist_storage(MCM)', axis=1, inplace=True)
    hist_cap.rename({'2055':'2055:=','week':' '},axis=1, inplace=True)
    new_cap.drop('new_storage(MCM)', axis=1, inplace=True)
    new_cap.rename({'2055':'2055:=','week':' '},axis=1, inplace=True)
    
    
    # Save files
    output_folder = os.path.join('processed_data', 'flood_protection_scenario', f'{level}%_change_buffer')
    os.makedirs(output_folder, exist_ok = True)

    file_path=os.path.join(output_folder, f'Fierza_at_{level}%buffer.xlsx')
    writer = pd.ExcelWriter(file_path, engine='xlsxwriter')
    
    
    _df.round(2).to_excel(writer, index=False, sheet_name='all')
    hist_cap.round(2).to_excel(writer, index=False, sheet_name='hist_cap')
    new_cap.round(2).to_excel(writer, index=False, sheet_name='new_cap')
    hist_x.round(2).to_excel(writer, index=False, sheet_name='hist_x')
    new_x.round(2).to_excel(writer, index=False, sheet_name='new_x')
    rule.round(2).to_excel(writer, index=False, sheet_name='rule')

    writer.save()
    


## 2.Spilje reservoir calculations:

In [40]:
#maping the monthly water level
hist_level = dict(df_hisopr_sp[['month', 'hist_level(m)']].values)
df_newopr_sp['hist_level(m)'] = df_newopr_sp['month'].round(0).map(hist_level)

# reading the storage volume based on the historical water levels
level_predict = np.poly1d(np.polyfit(spil_curve['Level(m)'], spil_curve['Volume(MCM)'], 2))
df_newopr_sp['hist_storage(MCM)'] = level_predict(df_newopr_sp['hist_level(m)'].round(0))

In [51]:
df_newopr_sp['max_level(m)'] = 585
df_newopr_sp['max_storage(MCM)'] = 520
df_newopr_sp['lowerlevel(m)'] = 485.75 # this is the turbine elevation
df_newopr_sp['hist_buffer(MCM)'] = df_newopr_sp['max_storage(MCM)'] - df_newopr_sp['hist_storage(MCM)']

In [54]:
levels = [5, 10, 15, 20]
Fierza_power = 500
spilje_power = 84
density = 1000 #(kg/m3) = 
g = 9.81 #(m/sec2) 
Turb_eff = 0.9 #
#fierza_lower_level = 176
#spilje_lower_level = 485.75 # this is the turbine elevation

for level in levels:  
    # Create the new dataframe
    _df = df_newopr_sp[['month', 'week', 'max_level(m)','hist_level(m)', 'max_storage(MCM)', 'hist_storage(MCM)','hist_buffer(MCM)','lowerlevel(m)']].copy()
    
    # Calculating new buffer and storage
    _df['new_buffer(MCM)'] = _df['hist_buffer(MCM)'] + (_df['hist_buffer(MCM)']*(level/100))
    _df.loc[(_df['month']>4) & (_df['month']<10), 'new_buffer(MCM)'] = _df['hist_buffer(MCM)']
    
    _df['new_storage(MCM)'] = _df['max_storage(MCM)'] - _df['new_buffer(MCM)']
    
    _df['changeinbuffer(MCM)'] = (_df['new_buffer(MCM)'] - _df['hist_buffer(MCM)'])/_df['hist_buffer(MCM)']
    _df['additional_buffer(MCM)'] = (_df['new_buffer(MCM)'] - _df['hist_buffer(MCM)'])
    
    # Reading the new water level based on the volume-elevation curve
    level_predict = np.poly1d(np.polyfit(spil_curve['Volume(MCM)'], spil_curve['Level(m)'], 3))
    _df['new_level(m)'] = level_predict(_df['new_storage(MCM)'].round(0))
    _df.loc[(_df['month']>4) & (_df['month']<10), 'new_level(m)'] = _df['hist_level(m)']
    
    #_df['lowerlevel(m)'] = spilje_lower_level
    
    # Add the calculations of X for each week
    _df['hist_head(m)'] = _df['hist_level(m)'] - _df['lowerlevel(m)']
    _df['hist_Q(m/sec)'] = ((spilje_power*1000000)/(density*g*Turb_eff*_df['hist_head(m)']))
    _df['hist_flow(MCM)'] = (_df['hist_Q(m/sec)']*3600*8760)/1000000
    _df['hist_energy(PJ)'] = ((spilje_power/1000)*31.536)
    _df['hist_X(MCM/PJ)'] = _df['hist_flow(MCM)']/_df['hist_energy(PJ)']
    
    _df['new_head(m)'] = _df['new_level(m)'] - _df['lowerlevel(m)']
    _df['new_Q(m/sec)'] = ((spilje_power*1000000)/(density*g*Turb_eff*_df['new_head(m)']))
    _df['new_flow(MCM)'] = (_df['new_Q(m/sec)']*3600*8760)/1000000
    _df['new_energy(PJ)'] = ((spilje_power/1000)*31.536)
    _df['new_X(MCM/PJ)'] = _df['new_flow(MCM)']/_df['new_energy(PJ)']
    _df['mode'] = 0
    
    # Generate the outputfile that goes to osemosys
    hist_cap = _df[['month','week','hist_storage(MCM)']]
    new_cap = _df[['month','week','new_storage(MCM)']]
    hist_x = _df[['week','mode','hist_X(MCM/PJ)']]
    new_x = _df[['week','mode','new_X(MCM/PJ)']]
    rule = _df.groupby('month')[['hist_level(m)','new_level(m)','additional_buffer(MCM)']].mean().round(1).reset_index()
    rule['diff'] = rule['hist_level(m)'] - rule['new_level(m)']
    
    #res_cap = ref[['week','Utilized Volume (MCM)']]
    for i in range(20,56):
        hist_cap['20{}'.format(i)]=hist_cap['hist_storage(MCM)']
        new_cap['20{}'.format(i)]=new_cap['new_storage(MCM)']
    hist_cap.drop('hist_storage(MCM)', axis=1, inplace=True)
    hist_cap.rename({'2055':'2055:=','week':' '},axis=1, inplace=True)
    new_cap.drop('new_storage(MCM)', axis=1, inplace=True)
    new_cap.rename({'2055':'2055:=','week':' '},axis=1, inplace=True)
    
    
    # Save files
    output_folder = os.path.join('processed_data', 'flood_protection_scenario', f'{level}%_change_buffer')
    os.makedirs(output_folder, exist_ok = True)

    file_path2=os.path.join(output_folder, f'Spilje_at_{level}%buffer.xlsx')
    writer = pd.ExcelWriter(file_path2, engine='xlsxwriter')
    
    
    _df.round(2).to_excel(writer, index=False, sheet_name='all')
    hist_cap.round(2).to_excel(writer, index=False, sheet_name='hist_cap')
    new_cap.round(2).to_excel(writer, index=False, sheet_name='new_cap')
    hist_x.round(2).to_excel(writer, index=False, sheet_name='hist_x')
    new_x.round(2).to_excel(writer, index=False, sheet_name='new_x')
    rule.round(2).to_excel(writer, index=False, sheet_name='rule')

    writer.save()
    

# End